In [12]:
import open3d as o3d
import numpy as np

def calculate_distance(plane1, plane2):
    # 두 평면 사이의 거리를 계산합니다.
    norm = np.linalg.norm(np.array(plane1[:3]))  # 평면의 법선 벡터 크기 계산
    distance = abs(plane2[3] - plane1[3]) / norm  # 두 평면 사이의 거리 계산
    return distance

# 포인트 클라우드 데이터를 로드합니다.
pcd = o3d.io.read_point_cloud("C:/Users/rjhyu/파이썬 연습/2024 new/230727_20mm_45,60 1.pts")

# RANSAC을 사용하여 첫 번째 평면을 찾습니다.
plane1_model, inliers1 = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud1 = pcd.select_by_index(inliers1)

# 첫 번째 평면의 인라이어를 제거합니다.
outlier_cloud = pcd.select_by_index(inliers1, invert=True)

# RANSAC을 사용하여 두 번째 평면을 찾습니다.
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud2 = outlier_cloud.select_by_index(inliers2)

# 두 평면 사이의 최소 거리를 계산합니다.
distance = calculate_distance(plane1_model, plane2_model)
print(f"The minimum distance between the two planes is: {distance} units")

# 시각화
inlier_cloud1.paint_uniform_color([1.0, 0, 0])  # 첫 번째 평면은 빨간색
inlier_cloud2.paint_uniform_color([0, 1.0, 0])  # 두 번째 평면은 초록색
o3d.visualization.draw_geometries([inlier_cloud1, inlier_cloud2])

# 계산된 거리와 포인트 클라우드를 저장할 파일의 경로를 설정합니다.
distance_file_path = "C:/Users/rjhyu/파이썬 연습/2024 new/calculated_distance.txt"
inlier_cloud1_path = "C:/Users/rjhyu/파이썬 연습/2024 new/inlier_cloud1.pts"
inlier_cloud2_path = "C:/Users/rjhyu/파이썬 연습/2024 new/inlier_cloud2.pts"

# 계산된 거리를 텍스트 파일로 저장합니다.
with open(distance_file_path, "w") as file:
    file.write(f"The minimum distance between the two planes is: {distance} units\n")

# 포인트 클라우드(inliers)를 파일로 저장합니다.
o3d.io.write_point_cloud(inlier_cloud1_path, inlier_cloud1)
o3d.io.write_point_cloud(inlier_cloud2_path, inlier_cloud2)


[Open3D WARNING] Read PTS: only points and colors attributes are supported.
The minimum distance between the two planes is: 2.286944794278593 units


True

In [1]:
# 필요한 라이브러리를 불러옵니다
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.linear_model import RANSACRegressor

# 데이터를 불러옵니다
points = np.loadtxt("C:/Users/rjhyu/파이썬 연습/2024 new/230727_20mm_C0 1.pts", skiprows=1)

# 데이터 전처리를 수행합니다
# 예: 아웃라이어 제거, 필터링, 등등

# RANSAC 등을 사용하여 평면을 검출합니다
ransac = RANSACRegressor()
ransac.fit(points[:, :2], points[:, 2])  # x, y 좌표를 기반으로 z 좌표를 학습

# 클러스터링을 수행하여 각 벽돌을 구분합니다
dbscan = DBSCAN(eps=0.05, min_samples=10)
clusters = dbscan.fit_predict(points)

# 각 클러스터(벽돌)의 중심점을 계산합니다
centers = []
for label in set(clusters):
    if label != -1:  # -1은 노이즈를 나타냅니다
        center = points[clusters == label].mean(axis=0)
        centers.append(center)

# 두 클러스터(벽돌)의 중심점 사이의 거리를 계산합니다
distance = np.linalg.norm(centers[0] - centers[1])  # 이 부분은 실제 상황에 맞게 조정 필요

# 결과를 출력합니다
print("이격 거리:", distance)


이격 거리: 1.0000365609845399


In [8]:
import numpy as np
import open3d as o3d

# 데이터 로딩
data = np.loadtxt("C:/Users/rjhyu/파이썬 연습/2024 new/230727_20mm_45,60-2.txt")

# Open3D PointCloud 객체 생성
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data[:,:3])  # x, y, z 좌표 추정

# 필요한 처리 수행 (예: RANSAC을 사용한 평면 검출 등)
# RANSAC을 사용하여 첫 번째 평면을 찾습니다.
plane1_model, inliers1 = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud1 = pcd.select_by_index(inliers1)

# 첫 번째 평면의 인라이어를 제거합니다.
outlier_cloud = pcd.select_by_index(inliers1, invert=True)

# RANSAC을 사용하여 두 번째 평면을 찾습니다.
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud2 = outlier_cloud.select_by_index(inliers2)

# 두 평면 사이의 최소 거리를 계산합니다.
distance = calculate_distance(plane1_model, plane2_model)
print(f"The minimum distance between the two planes is: {distance} units")

# 시각화
inlier_cloud1.paint_uniform_color([1.0, 0, 0])  # 첫 번째 평면은 빨간색
inlier_cloud2.paint_uniform_color([0, 1.0, 0])  # 두 번째 평면은 초록색
o3d.visualization.draw_geometries([inlier_cloud1, inlier_cloud2])

# 계산된 거리와 포인트 클라우드를 저장할 파일의 경로를 설정합니다.
distance_file_path = "C:/Users/rjhyu/파이썬 연습/2024 new/calculated_distance.txt"
inlier_cloud1_path = "C:/Users/rjhyu/파이썬 연습/2024 new/inlier_cloud1.pts"
inlier_cloud2_path = "C:/Users/rjhyu/파이썬 연습/2024 new/inlier_cloud2.pts"

# 계산된 거리를 텍스트 파일로 저장합니다.
with open(distance_file_path, "w") as file:
    file.write(f"The minimum distance between the two planes is: {distance} units\n")

# 포인트 클라우드(inliers)를 파일로 저장합니다.
o3d.io.write_point_cloud(inlier_cloud1_path, inlier_cloud1)
o3d.io.write_point_cloud(inlier_cloud2_path, inlier_cloud2)

The minimum distance between the two planes is: 0.32229319474598356 units
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 요청한 변환 작업을 지원할 수 없습니다. 


True